In [1]:
## DRIVE ACCESS
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir("drive/My Drive/Colab Notebooks/KTJ2020")
!ls

atul_files		    MapMyIndia-Code_STEP2.ipynb
distance_from_start.csv     MapMyIndia_RestAPI_dataRetrieval_STEP1.ipynb
distance_matrix.csv	    Petrol-Pumps.csv
distance_matrix.gsheet	    sample_marker.jpg
InteractiveAPI_STEP3.ipynb  sample_polyline.jpg


In [0]:
# ## READ CSV (lat,long,profit)
# import pandas as pd
# col_name = ["Index","Latitute","Longitute"]
# coords = pd.read_csv("Petrol-Pumps.csv", index_col=0, header=None,delimiter=";", names=col_name)
# coords.head()
# # coords.to_csv("Petrol-Pumps_new.csv")
# # coords = pd.read_csv("Petrol-Pumps_new.csv", index_col=0, header=0)
# # coords.head()

# ## ADD PROFIT TO CSV
# import random
# profits = [random.randrange(1, 100, 1) for i in range(coords.shape[0])]
# coords["Profit"]=profits
# # coords.drop(labels=None,columns=Index)
# coords.to_csv("Petrol-Pumps_new.csv",index=False)
# coords = pd.read_csv("Petrol-Pumps_new.csv")
# coords.head()

In [4]:
import pandas as pd
coords = pd.read_csv("Petrol-Pumps.csv")
coords.head()

,Latitute,Longitute,Profit
0,28.746731,77.151751,48
1,28.544869,77.128134,69
2,28.698757,77.292577,17
3,28.675332,77.225378,40
4,28.597172,77.195823,92


In [5]:
coords = coords[(coords["Latitute"]<28.82) & (coords["Latitute"]>27.5) & (coords["Longitute"]>77)]
coords.shape[0]

117

In [0]:
# coords.to_csv("Petrol-Pumps_new.csv",index=False)

In [0]:
### RUN OPTIMIZATION CODE BY ATUL ###

In [0]:
rest_api_key = "pjonp7jywyzz2hxqiex7lrls1vqkmjfn/"
interactive_map_api_key = "e3b3l6eu4dt57q7o5hu2ks7knw5mfvl6/"

rest_api_key_at = "jry2mpzyiykhd78e23mmjxr4nc59kwhf/"
interactive_map_api_key_at = "zggxhs8yqrrfabtcprxk111no5w9ddzo/"

In [0]:
import requests
def make_request(url,body):
  r = requests.get(url = url+body)
  return r

In [0]:
def get_Routing(startLat,startLong, endLat,endLong):
  
  # for index in order:
  #   lati=coords.iloc[index,0]
  #   longi = coords.iloc[index,1]
  #   body+=str(longi)+","+str(lati)+";"

  url = "https://apis.mapmyindia.com/advancedmaps/v1/"+rest_api_key+"route_adv/driving/"
  body=str(startLong)+","+str(startLat)+";"


  body+=str(endLong)+","+str(endLat)
  print(url+body)

  # points = []
  lats=[]
  longs=[]

  response = make_request(url,body)
  if response.status_code == 200:
    y=response.json()
    wayPoints = y["waypoints"]#["legs"]
    print(wayPoints)

  for point in wayPoints:
    if point==None:
      continue
    pt = {"longitute":point["location"][0],"latitute":point["location"][1]}
    longs.append(point["location"][0])
    lats.append(point["location"][1])
    print(pt)
    # points.append(pt)
  
  return lats,longs

In [0]:
def create_polyline_url(order, startLat, startLong):
  url = "http://apis.mapmyindia.com/advancedmaps/v1/"+interactive_map_api_key+"still_image_polyline?"
  body = "height=500&width=480&polyline=[["+str(startLong)+","+str(startLat)+"],["

  prevLat = startLat
  prevLong = startLong

  for index in order:
    lati=coords.iloc[index,0]
    longi = coords.iloc[index,1]
    body+=str(longi)+","+str(lati)+"],["
    lats,longs = get_Routing(lati,longi,prevLat,prevLong)
    if len(longs)==0:
      continue
    for i in range(len(longs)):
      body+=str(longs[i])+","+str(lats[i])+"],["


    prevLat = lati
    prevLong = longi
  
  body=body[:-2]
  body+="]&color=#ff00f7&offset=10,44&padding_x=50&padding_y=100"
  print(url+body)
  print(body)
  response = make_request(url,body)
  if response.status_code == 200:
    with open("sample_polyline.jpg", 'wb') as f:
        f.write(response.content)
    print("[INFO] file saved.")
  else:
    print("Error: "+str(response.status_code))

In [0]:
def create_marker_url(order, startLat, startLong):
  url = "http://apis.mapmyindia.com/advancedmaps/v1/"+interactive_map_api_key+"still_image?center="
  body = ""
  lati_avg = startLat
  longi_avg = startLong
  count = 1
  for index in order:
    count+=1
    lati=coords.iloc[index,0]
    longi = coords.iloc[index,1]
    body+=str(lati)+","+str(longi)+"|"
    lati_avg+=lati
    longi_avg+=longi

  body=body[:-1]
  body+=">"
  lati_avg = lati_avg/count
  longi_avg = longi_avg/count
  url+=str(lati_avg)+","+str(longi_avg)+"&zoom=10&size=500x480&ssf=1&markers="+str(startLat)+","+str(startLong)+"|"

  print(url)
  print(body)
  
  response = make_request(url,body)
  if response.status_code == 200:
    with open("sample_marker.jpg", 'wb') as f:
        f.write(response.content)
    print("[INFO] file saved.")
  else:
    print(response.status_code)

In [43]:
order = [19, 13, 40, 41, 22, 21, 29, 1] #[20, 30, 28, 45, 8, 25, 33, 6, 47, 15, 34, 48, 9, 35, 16, 49]
r_p = create_polyline_url(order,28.80,77.35)
# r_m = create_marker_url(order,28.80,77.35)

https://apis.mapmyindia.com/advancedmaps/v1/pjonp7jywyzz2hxqiex7lrls1vqkmjfn/route_adv/driving/77.08648859999994,28.5288717;77.35,28.8
[{'hint': 'I1zmgf___38IAAAALwAAACMAAAAAAAAAODfsQB6NBUIBNu9BAAAAAAgAAAAvAAAAIwAAAAAAAABEAAAAoj-YBKpQswEZP5gE6FCzAQEAvwEUFRQe', 'distance': 15.067802, 'location': [77.086626, 28.52881], 'name': 'Old Delhi Gurgaon Road'}, {'hint': 'M0f5gDlH-YBtAAAABwAAAC4CAAAAAAAAEw2YQmuSi0BaasFDAAAAAG0AAAAHAAAALgIAAAAAAABEAAAAEk-cBE9ttwFwRJwEAHS3AQUAfw0UFRQe', 'distance': 326.642406, 'location': [77.352722, 28.798287], 'name': ''}]
{'longitute': 77.086626, 'latitute': 28.52881}
{'longitute': 77.352722, 'latitute': 28.798287}
https://apis.mapmyindia.com/advancedmaps/v1/pjonp7jywyzz2hxqiex7lrls1vqkmjfn/route_adv/driving/77.08783849999998,28.621899;77.08648859999994,28.5288717
[{'hint': 'IycBgP___38rAAAANAAAAAAAAAAAAAAA8863QbIqjkAAAAAAAAAAACsAAAA0AAAAAAAAAAAAAABEAAAAWUSYBEu8tAFeRJgES7y0AQAAfwsUFRQe', 'distance': 0.488952, 'location': [77.087833, 28.621899], 'name': 'Professo

In [0]:
def roadBWpoints(order,startLat,startLong):
  
  for index in order:
    lati=coords.iloc[index,0]
    longi = coords.iloc[index,1]
    body+=str(longi)+","+str(lati)+";"

  url = "https://apis.mapmyindia.com/advancedmaps/v1/"+rest_api_key+"snapToRoad?pts="
  body=str(startLong)+","+str(startLat)+";"


  body+=str(startLong)+","+str(startLat)
  print(url+body)

  points = []

  response = make_request(url,body)
  if response.status_code == 200:
    y=response.json()
    wayPoints = y["results"]["snappedPoints"]
    print(wayPoints)

  for point in wayPoints:
    if point==None:
      continue
    point = {"longitute":point["location"][0],"latitute":point["location"][1]}
    print(point)
    points.append(point)
  
  return points

In [14]:
r_road = roadBWpoints(order,28.80,77.35)

https://apis.mapmyindia.com/advancedmaps/v1/jry2mpzyiykhd78e23mmjxr4nc59kwhf/snapToRoad?pts=77.35,28.8;77.08648859999994,28.5288717;77.08783849999998,28.621899;77.06197700000007,28.566214;77.08648859999994,28.5288717;77.17754009999999,28.5232298;77.05978700000003,28.560096;77.09995779999997,28.5561624;77.12813449999999,28.5448686;77.35,28.8


UnboundLocalError: ignored